pmgus_dev_stuff.ipynb

get pre-market price data for SOFI
-> do i need a websocket for pre-market data? after hours, post market ours data?
get pre-market clone data of TradingView
add features new and other ideas that we find

In [5]:
import pandas as pd
import numpy as np
import requests
import os
import json
import yfinance as yf
import datetime
from datetime import datetime, timedelta
from typing import List, Dict, Any

In [6]:
import dotenv
from dotenv import load_dotenv
load_dotenv()
pg_api_key = os.getenv('polygon_api_key')

True

In [235]:
# initial automation with data feed
## streamlit, and cool to use lightweight-charts with websockets?
### flashing web sockets not static snapshots, but snapshots also
#### up to 0929est, 0629pst

In [236]:
# return all stocks listed on the NYSE or NASDAQ that have pre-market price $ movement greater that $0.00. 
## in other words no negative price movement
### filter out this macro first filter and ingest the stock ticker symbols just to a list simply or more other stock data?
#### return a list of ticket symbols that meet the criteria

In [8]:

def get_premarket_gainers() -> List[str]:
    api_key = os.getenv('polygon_api_key')

    """
    Fetches stocks from NYSE and NASDAQ with positive pre-market price movement.
    
    Returns:
        List[str]: List of ticker symbols meeting the criteria
    """
    # Base URL for Polygon.io API
    base_url = "https://api.polygon.io/v2/snapshot/locale/us/markets/stocks/tickers"
    
    # Parameters for the API request
    params = {
        'apiKey': api_key,
        'exchange': 'NYSE,NASDAQ',  # Filter for NYSE and NASDAQ listed stocks
        'market': 'stocks',
        'active': True
    }
    
    try:
        # Make API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        
        # Filter for stocks with positive pre-market movement
        gainers = []
        for ticker in data.get('tickers', []):
            # Get pre-market data
            pre_market_change = ticker.get('todaysChange', 0)
            
            # Check if pre-market change is positive
            if pre_market_change > 0:
                gainers.append(ticker['ticker'])
        
        return gainers
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

In [9]:
# execute and find the largest group of gainers
pm_gainers_ticker_list = get_premarket_gainers()

print(f"Found {len(pm_gainers_ticker_list)} stocks with positive pre-market movement")
print("\nGaining Tickers:")
print(", ".join(pm_gainers_ticker_list))

Found 731 stocks with positive pre-market movement

Gaining Tickers:
ETHE, TRMB, COIN, TER, PNFP, RLX, SES.WS, PFIS, TLRY, QSI, SIGI, PI, BBW, ALB, RCAT, TOL, ERIE, QMMM, BNAI, BVFL, MTB, RLI, OPTX, RVSN, EVTL, GWRE, JL, TTE, JAGX, GMAB, HSII, THG, HII, CPT, PKG, CADE, CLH, RGTI, NA, NCNO, AIN, LPSN, ETH, RITM, ZK, PRA, TREX, GRFX, PDYN, HCI, MSTX, APLE, NGNE, PHUN, LPTH, MGOL, ASML, ETHU, QS, AME, PAYS, PXLW, POAI, CADL, LYTS, CBZ, NXRT, COMP, STI, ADTX, BECN, FCN, YANG, ZG, HQY, LODE, WULF, CAPT, NETZ, KITT, HBANP, SLB, SLDP, BHAT, SHV, SRM, BWLP, APPF, SPXS, NOVT, GPI, MVST, ITA, BXSL, EXP, CFR, UVE, WIMI, KSCP, WKHS, MULN, NKGN, BKTI, SRFM, VXUS, ELEV, FIZZ, EQNR, APGE, TSLQ, CETX, BBSI, BELFB, ACLX, OMH, BTDR, LDTC, SSBK, CYD, VZ, AILE, VIG, CMCT, VRAX, CANF, LAAC, LABD, CNXC, REKR, LBRDK, QTUM, AFL, ITP, MARA, ELPW, NVDQ, GTLB, NOTE, ANTE, AZTA, ZKIN, LOVE, PWM, CONY, CGC, NTNX, XOVR, KULR, CACC, KHC, SAM, ELMD, HOLO, LUNR, ROG, VSAT, LTRY, AISP, ABEV, VNET, WMB, IRBT, PONY, EETH

In [237]:
# # my test tickers
# ticker_list = ['SOFI','PLTR','NOW','ROIC']


# POLYGONIO

In [238]:
# #### PolygonIO DEVELOPEMENT ####
# def get_snapshot(ticker, api_key):
#     url = f"https://api.polygon.io/v2/snapshot/locale/us/markets/stocks/tickers/{ticker}?apiKey={api_key}"
#     response = requests.get(url)
    
#     if response.status_code == 200:
#         data = response.json()
#         return data
#     else:
#         print(f"Error: {response.status_code} - {response.text}")
#         return None

In [239]:
# #### PolygonIo DEVELOPEMENT ####
# ticker = ticker_list[0]
# api_key = os.getenv("polygon_api_key")  
# snapshot_data = get_snapshot(ticker, api_key)

# # Print the raw JSON response in a nice format
# if snapshot_data:
#     print(json.dumps(snapshot_data, indent=2))
    
#     # If you want to extract specific fields, for example the last trade price:
#     if 'ticker' in snapshot_data and 'lastTrade' in snapshot_data['ticker']:
#         last_trade_price = snapshot_data['ticker']['lastTrade'].get('p')
#         print(f"\nThe most recent trade price for {ticker} is: {last_trade_price}")
#     else:
#         print("\nNo 'lastTrade' data found in the snapshot response.")

In [12]:
def get_enhanced_snapshot(ticker, api_key):
    base_url = "https://api.polygon.io/v2"
    snapshot_url = f"{base_url}/snapshot/locale/us/markets/stocks/tickers/{ticker}"
    # Get previous day's close and pre-market data
    aggs_url = f"{base_url}/aggs/ticker/{ticker}/range/1/minute/previous"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    try:
        snapshot_response = requests.get(snapshot_url, headers=headers)
        snapshot_response.raise_for_status()
        snapshot_data = snapshot_response.json()
        
        # Calculate pre-market metrics
        current_price = snapshot_data['ticker']['lastTrade']['p']
        prev_close = snapshot_data['ticker']['prevDay']['c']
        
        # Calculate pre-market changes
        pm_change = current_price - prev_close
        pm_change_percent = (pm_change / prev_close) * 100
        
        # Enhance the snapshot data with pre-market metrics
        enhanced_data = {
            'Symbol': ticker,
            'prev_close': prev_close,
            'pm_change': pm_change,
            # volume review this
            'volume': snapshot_data['ticker']['day']['v'],
            'vwap': snapshot_data['ticker']['day']['vw'],
            'timestamp': snapshot_data['ticker']['updated']
        }
        
        return enhanced_data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

In [14]:
def build_screening_df(tickers, api_key):

    results = []

    for ticker in tickers:
        data = get_enhanced_snapshot(ticker, api_key)
        if data:
            results.append(data)
    
    df = pd.DataFrame(results)
    # volume review this
    # Add additional calculated columns
    if not df.empty:
        df['rel_volume'] = df['volume'] / df['volume'].mean()
        # YOU CAN ADD STUFF HERE LATER        
    return df

In [15]:
# build screening dataframe
pg_api_key = os.getenv("polygon_api_key")  
polygon_df = build_screening_df(pm_gainers_ticker_list, pg_api_key)
display(polygon_df)

,Symbol,prev_close,pm_change,volume,vwap,timestamp,rel_volume
0,ETHE,27.900,0.770,0,0,1735293600000000000,NaN
1,TRMB,72.300,0.610,0,0,1735292100000000000,NaN
2,COIN,274.410,0.470,0,0,1735293540000000000,NaN
3,TER,130.840,0.010,0,0,1735290060000000000,NaN
4,PNFP,115.650,0.650,0,0,1735292880000000000,NaN
...,...,...,...,...,...,...,...
726,HOV,134.530,0.970,0,0,1735292280000000000,NaN
727,FHI,41.510,0.010,0,0,1735290180000000000,NaN
728,EVAX,0.846,0.554,0,0,1735293801791298307,NaN
729,ADUS,130.180,5.820,0,0,1735290420000000000,NaN


# Yahoo Finance

In [ ]:
# #### Yahoo DEVELOPEMENT ####
# print(ticker_list[1])
# ticker = yf.Ticker(ticker_list[1])
# info = ticker.info
# print(f"Ticker: {ticker_list[1]}")
# # Print all the keys in the info dictionary (useful for discovering other available data)
# print("\nTicker Info Keys:", info.keys())
# # Print float shares and shares outstanding if they exist in the info dict
# print("\nFloat Shares:", info.get('floatShares'))
# print("Shares Outstanding:", info.get('sharesOutstanding'))

In [ ]:
# List of tickers to fetch data for
# print(ticker_list)
# This list will hold dictionaries of our extracted data
data_rows = []

for symbol in pm_gainers_ticker_list:
    ticker_obj = yf.Ticker(symbol)
    info = ticker_obj.info
    
    # Build a row of data
    row = {
        "Symbol": symbol,  # Or you could use info.get("symbol") if you trust Yahoo's symbol field
        "FloatShares": info.get('floatShares'),
        "Float shares outstanding": info.get('sharesOutstanding'),
        # Add other fields here if needed
        'Industry': info.get('industry'),   
        'Sector': info.get('sector'),
        'Market capitalization': info.get('marketCap'),
        'averageDailyVolume10Day': info.get('averageDailyVolume10Day'),
        'sharesShort': info.get('sharesShort'),
        'shortRatio': info.get('shortRatio'),}
    data_rows.append(row)

# Convert the list of dictionaries into a pandas DataFrame
yahoo_df = pd.DataFrame(data_rows)
display(yahoo_df)

In [245]:
# VOLATILITY metrics calculation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests

class MarketMetricsCalculator:
    def __init__(self, polygon_api_key):
        self.polygon_api_key = polygon_api_key
        self.base_url = "https://api.polygon.io/v2"
        self.headers = {
            "Authorization": f"Bearer {polygon_api_key}",
            "Content-Type": "application/json"
        }

    def get_historical_aggregates(self, ticker, from_date, to_date, multiplier=1, timespan="minute"):
        """Get historical aggregates from Polygon.io"""
        url = f"{self.base_url}/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
        response = requests.get(url, headers=self.headers)
        return response.json() if response.status_code == 200 else None

    def calculate_volatility(self, prices, window):
        """Calculate rolling volatility"""
        returns = np.log(prices / prices.shift(1))
        return returns.rolling(window=window).std() * np.sqrt(252)

    def get_market_metrics(self, ticker):
        """Calculate comprehensive market metrics"""
        today = datetime.now()
        one_month_ago = (today - timedelta(days=30)).strftime('%Y-%m-%d')
        today_str = today.strftime('%Y-%m-%d')

        # Get current snapshot
        snapshot_url = f"{self.base_url}/snapshot/locale/us/markets/stocks/tickers/{ticker}"
        snapshot_response = requests.get(snapshot_url, headers=self.headers)
        snapshot_data = snapshot_response.json() if snapshot_response.status_code == 200 else None

        # Get historical data
        historical_data = self.get_historical_aggregates(ticker, one_month_ago, today_str)

        if not snapshot_data or not historical_data or 'results' not in historical_data:
            return None

        # Current data from snapshot
        current_price = snapshot_data['ticker']['lastTrade']['p']
        prev_close = snapshot_data['ticker']['prevDay']['c']
        current_volume = snapshot_data['ticker']['day']['v']
        vwap = snapshot_data['ticker']['day']['vw']

        # Calculate pre-market metrics
        pre_market_change_pct = ((current_price - prev_close) / prev_close) * 100
        pre_market_gap = ((snapshot_data['ticker']['day']['o'] - prev_close) / prev_close) * 100

        # Process historical data for volatility and volume metrics
        df = pd.DataFrame(historical_data['results'])
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        df.set_index('t', inplace=True)

        # Calculate volatilities
        prices = df['c']
        vol_1d = self.calculate_volatility(prices, 1).iloc[-1]
        vol_1w = self.calculate_volatility(prices, 5).iloc[-1]
        vol_1m = self.calculate_volatility(prices, 22).iloc[-1]

        # Calculate volume metrics
        avg_volume = df['v'].mean()
        rel_volume_1d = current_volume / avg_volume if avg_volume > 0 else 0

        # Calculate relative volume at time
        current_time = datetime.now().time()
        historical_volume_at_time = df[df.index.time <= current_time]['v'].mean()
        rel_volume_at_time = current_volume / historical_volume_at_time if historical_volume_at_time > 0 else 0

        return {
            'Symbol': ticker,
            'Relative Volume 1 day': rel_volume_1d,
            'Relative Volume at Time': rel_volume_at_time,
            'Pre-market Change %': pre_market_change_pct,
            'Pre-market Gap %': pre_market_gap,
            'Price': current_price,
            'Volume Weighted Average Price 1 day': vwap,
            'Volatility 1 day': vol_1d,
            'Volatility 1 week': vol_1w,
            'Volatility 1 month': vol_1m,
            'Pre-market Volume': current_volume
        }

    def build_metrics_df(self, ticker_list):
        """Build DataFrame with market metrics for multiple tickers"""
        results = []
        for ticker in ticker_list:
            metrics = self.get_market_metrics(ticker)
            if metrics:
                results.append(metrics)
        
        return pd.DataFrame(results)

In [246]:
volatility_df = MarketMetricsCalculator(api_key)
volatility_df = volatility_df.build_metrics_df(ticker_list)

In [247]:
display(volatility_df)

,Symbol,Relative Volume 1 day,Relative Volume at Time,Pre-market Change %,Pre-market Gap %,Price,Volume Weighted Average Price 1 day,Volatility 1 day,Volatility 1 week,Volatility 1 month,Pre-market Volume
0,SOFI,651.626455,52845.407341,3.807740,-0.249688,16.63,16.4665,NaN,0.009986,0.024352,36892700.0
1,PLTR,817.581929,34747.498848,-0.679777,0.946832,81.82,82.3025,NaN,0.004879,0.007128,60321658.0
2,NOW,162.098325,0.000000,-0.593364,-0.502913,1099.00,1099.4303,NaN,0.008634,0.010458,425125.0
3,ROIC,109.788506,0.000000,-0.172612,-0.172612,17.35,17.3647,NaN,0.004930,0.003954,352225.0


In [248]:
# merge the THREE dataframes
pgyf_df = pd.merge(polygon_df, yahoo_df, on="Symbol", how="inner")
pgyfvt_df = pd.merge(pgyf_df, volatility_df, on="Symbol", how="inner")

print(len(pgyfvt_df.columns))
# print(pgyfvt_df.columns)
display(pgyfvt_df)

25


,Symbol,prev_close,pm_change,volume,vwap,timestamp,rel_volume,FloatShares,Float shares outstanding,Industry,...,Relative Volume 1 day,Relative Volume at Time,Pre-market Change %,Pre-market Gap %,Price,Volume Weighted Average Price 1 day,Volatility 1 day,Volatility 1 week,Volatility 1 month,Pre-market Volume
0,SOFI,16.02,0.61,36892700.0,16.4665,1735261200000000000,1.505952,1065212727,1085149952,Credit Services,...,651.626455,52845.407341,3.807740,-0.249688,16.63,16.4665,NaN,0.009986,0.024352,36892700.0
1,PLTR,82.38,-0.56,60321658.0,82.3025,1735261200000000000,2.462317,2097077792,2180649984,Software - Infrastructure,...,817.581929,34747.498848,-0.679777,0.946832,81.82,82.3025,NaN,0.004879,0.007128,60321658.0
2,NOW,1105.56,-6.56,425125.0,1099.4303,1735258920000000000,0.017354,205718681,206316992,Software - Application,...,162.098325,0.000000,-0.593364,-0.502913,1099.00,1099.4303,NaN,0.008634,0.010458,425125.0
3,ROIC,17.38,-0.03,352225.0,17.3647,1735246860000000000,0.014378,124391879,127788000,REIT - Retail,...,109.788506,0.000000,-0.172612,-0.172612,17.35,17.3647,NaN,0.004930,0.003954,352225.0


In [254]:
# numeric columns
pgyfvt_numeric_cols = ['Symbol',
'prev_close',
'pm_change',
'Price',
'Market capitalization',
'Pre-market Change %',
'Pre-market Gap %',
'Float shares outstanding',
'FloatShares',
# volume
'Relative Volume 1 day',
# 'volume', # same
'Relative Volume at Time',
'Volume Weighted Average Price 1 day',
# 'vwap', # same
'Pre-market Volume',
# volatility
'Volatility 1 day',
'Volatility 1 week', 
'Volatility 1 month']

# add OTHER columns
pgyfvt_other_cols = ['Industry','Sector']

# consolidated cols lists
pgyfvt_cols = pgyfvt_numeric_cols + pgyfvt_other_cols
print(len(pgyfvt_cols))
print(pgyfvt_cols)

# re-order cols
pgyfvt_df = pgyfvt_df[pgyfvt_cols]
display(pgyfvt_df.head(4))

18
['Symbol', 'prev_close', 'pm_change', 'Price', 'Market capitalization', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'FloatShares', 'Relative Volume 1 day', 'Relative Volume at Time', 'Volume Weighted Average Price 1 day', 'Pre-market Volume', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Industry', 'Sector']


,Symbol,prev_close,pm_change,Price,Market capitalization,Pre-market Change %,Pre-market Gap %,Float shares outstanding,FloatShares,Relative Volume 1 day,Relative Volume at Time,Volume Weighted Average Price 1 day,Pre-market Volume,Volatility 1 day,Volatility 1 week,Volatility 1 month,Industry,Sector
0,SOFI,16.02,0.61,16.63,18013489152,3.807740,-0.249688,1085149952,1065212727,651.626455,52845.407341,16.4665,36892700.0,NaN,0.009986,0.024352,Credit Services,Financial Services
1,PLTR,82.38,-0.56,81.82,187116568576,-0.679777,0.946832,2180649984,2097077792,817.581929,34747.498848,82.3025,60321658.0,NaN,0.004879,0.007128,Software - Infrastructure,Technology
2,NOW,1105.56,-6.56,1099.00,226765062144,-0.593364,-0.502913,206316992,205718681,162.098325,0.000000,1099.4303,425125.0,NaN,0.008634,0.010458,Software - Application,Technology
3,ROIC,17.38,-0.03,17.35,2217121792,-0.172612,-0.172612,127788000,124391879,109.788506,0.000000,17.3647,352225.0,NaN,0.004930,0.003954,REIT - Retail,Real Estate


In [252]:
# create market cap categories
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute
mcc_setup_df = categorize_market_cap(pgyfvt_df).copy()
# drop Undefined marketCapType
mcc_setup_df = mcc_setup_df[mcc_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
]

# apply conversion
mcc_setup_df = convert_columns_to_numeric(mcc_setup_df, numeric_columns)
display(mcc_setup_df)

,Symbol,prev_close,pm_change,Price,Market capitalization,Pre-market Change %,Pre-market Gap %,Float shares outstanding,FloatShares,Relative Volume 1 day,Relative Volume at Time,Volume Weighted Average Price 1 day,Pre-market Volume,Volatility 1 day,Volatility 1 week,Volatility 1 month,Industry,Sector,marketCapType
0,SOFI,16.02,0.61,16.63,18013489152,3.807740,-0.249688,1085149952,1065212727,651.626455,52845.407341,16.4665,36892700.0,NaN,0.009986,0.024352,Credit Services,Financial Services,Large caps
1,PLTR,82.38,-0.56,81.82,187116568576,-0.679777,0.946832,2180649984,2097077792,817.581929,34747.498848,82.3025,60321658.0,NaN,0.004879,0.007128,Software - Infrastructure,Technology,Large caps
2,NOW,1105.56,-6.56,1099.00,226765062144,-0.593364,-0.502913,206316992,205718681,162.098325,0.000000,1099.4303,425125.0,NaN,0.008634,0.010458,Software - Application,Technology,Titans
3,ROIC,17.38,-0.03,17.35,2217121792,-0.172612,-0.172612,127788000,124391879,109.788506,0.000000,17.3647,352225.0,NaN,0.004930,0.003954,REIT - Retail,Real Estate,Mid caps


In [253]:
# #### DEVELOPEMENT ####
# # base file path and file path setup
# base_dir_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/pmgus_inputs/' 
# tv_prefix = 'tv_screen_gap-up_'
# file_type_csv = '.csv'
# #####---------------------#####
# # screen_date = '2024-12-17' # worse day
# screen_date = '2024-12-18' # better day
# # screen_date = '2024-12-20' # not worse day
# #####---------------------#####
# tv_filename = base_dir_path + tv_prefix + screen_date + file_type_csv
# trading_view_df = pd.read_csv(tv_filename)
# # execute and print
# print(len(trading_view_df))
# new_tv_df = categorize_market_cap(trading_view_df).copy()
# display(new_tv_df.sample(5))